In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,clone_model
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import keras.backend as K

from utils import *

Using TensorFlow backend.


In [3]:
df = functions.parse_file(r"raw_data/EmoContext/train2.txt", "EmoContext")
df.head(5)

,id,label,turn1,turn2,turn3
0,0,others,dont worry ism girl,hmm how do i know if you are,whats ur name ? ?
1,1,angry,when did in i?,saw many times i think -_-,no . i never saw you .
2,2,others,by,by google chrome,where you live
3,3,angry,u r ridiculous,i might be ridiculous but i am telling the tru...,u little disgusting whore
4,4,others,just for time pass,wt do u do a a living then 4,maybe


In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
NR_WORDS = 8000
MAX_PROP_LENGTH = 128
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_train = tokenizer.texts_to_sequences(text_data)
X_train = pad_sequences(X_train, maxlen = MAX_PROP_LENGTH)

In [6]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_train = []

for idx,row in df.iterrows():
    Y_train.append(one_hot_vector(row['label']))

Y_train = np.array(Y_train)

In [7]:
df = functions.parse_file(r"raw_data/EmoContext/devwithlabels2.txt", "EmoContext")
df.head(5)

,id,label,turn1,turn2,turn3
0,0,angry,then dont ask me,youre a guy not as if you would understand,im not a guy fuck off
1,1,others,mixed things such as ??,the things you do . .,have you seen minions ??
2,2,happy,today ism very happy,and ism happy for you,i will be marry
3,3,others,noah bring me some,left it there oops,orb
4,4,others,it is thoo,i said soon master . .,he is pressuring me


In [8]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [9]:
#tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
#                                   lower=True,split=' ')

#tokenizer.fit_on_texts(text_data)

X_test = tokenizer.texts_to_sequences(text_data)
X_test = pad_sequences(X_test, maxlen = MAX_PROP_LENGTH)

In [10]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_test = []

for idx,row in df.iterrows():
    Y_test.append(one_hot_vector(row['label']))

Y_test = np.array(Y_test)

In [11]:
final_loaded_models = load_models()
merged_layer, input_layers = create_final_layers(final_loaded_models)
merged_layer = Dense(32)(merged_layer)
merged_layer = Dropout(0.5)(merged_layer)
merged_layer = Dense(4, activation='softmax')(merged_layer)
merged_model = Model(inputs=input_layers, output=merged_layer,name="merged_model")
merged_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy',f1])

ValueError: A `Concatenate` layer should be called on a list of at least 2 inputs

In [ ]:
merged_model.summary()

In [ ]:
mdcheck = ModelCheckpoint("trained_models/best_model_val_acc{val_acc:.4f}.h5", monitor='val_f1', save_best_only=True)

In [ ]:
batch_size = 64

Y_train = np.array(Y_train)
total = len(Y_train)
history = merged_model.fit(inputs, Y_train,
                    epochs=10, verbose=1, batch_size=batch_size,
                          shuffle=True)

In [ ]:
save_model(merged_model)

In [ ]:
df_test = functions.parse_file(r"raw_data/EmoContext/devwithlabels2.txt", "EmoContext")
df_test.head()

In [ ]:
text_data = []

for idx,row in df_test.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

res = merged_model.predict(X_test, batch_size=64, verbose=1)
res[3]

In [ ]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [ ]:
results = []
for r in res:
    results.append(softmax_convert(r))
    
df_test['label'] = results
df_test.head(50)
df_test.to_csv("combined_models_submission.txt",index=False , sep="\t")